In [ ]:
import os
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import librosa

# Set the number of OpenMP threads to 2
os.environ["OMP_NUM_THREADS"] = "2"

# Load a song and its corresponding data
random_song_id = 37
audio_file = f'../data/audio_files/processed/{random_song_id}.mp3'
df = pd.read_csv('../data/dataframes/clean_labeled.csv')
data = df.loc[df['SongID'] == random_song_id]

# Load audio file and separate harmonic and percussive components
y, sr = librosa.load(audio_file, sr=None)
y_perc = librosa.effects.hpss(y)[1]
onset_env = librosa.onset.onset_strength(y=y_perc, sr=sr, hop_length=512)


def filter_df(self, df):
    from scipy.signal import butter, filtfilt
    b, a = butter(2, 0.4)
    filtered_df = filtfilt(b, a, df)
    return filtered_df


def post_process_rcf(rcf):
    EPS = 1e-9  # Small epsilon to avoid division by zero
    adaptive_threshold(rcf)
    rcf_sum = np.sum(rcf) + EPS
    rcf = rcf / rcf_sum  # Normalize rcf to sum to unity




def viterbi_decode(self, rcfmat, wv):
    # This function should implement the Viterbi decoding algorithm.
    # Placeholder for simplicity. A real implementation is required for practical use.
    beat_period = np.argmax(rcfmat, axis=1)
    tempi = (60.0 * self.sample_rate / self.df_increment) / beat_period
    return beat_period, tempi

def calculate_beat_period(self, df, input_tempo=120.0, constrain_tempo=False):
    ray_param = (60 * self.sample_rate / 512) / input_tempo
    wv_len = 128
    if constrain_tempo:
        wv = np.exp(-((np.arange(wv_len) - ray_param) ** 2) / (2 * (ray_param / 4) ** 2))
    else:
        wv = (np.arange(wv_len) / ray_param ** 2) * np.exp(-((np.arange(wv_len)) ** 2) / (2 * ray_param ** 2))

    # Assuming df is a detection function array
    # This is a simplified placeholder implementation
    rcfmat = np.array([self.get_rcf(df, wv) for _ in range(len(df) // self.df_increment)])
    beat_period, tempi = self.viterbi_decode(rcfmat, wv)
    return beat_period, tempi

def calculate_beats(self, df, beat_period, alpha=0.9, tightness=4.0):
    # Placeholder for beat calculation using dynamic programming
    # A real implementation would be more complex and depend on the specific algorithm requirements.
    beats = np.arange(len(df))[::int(np.mean(beat_period))]
    return beats

# get_rcf function

In [ ]:
def adaptive_threshold(signal):
    # A simplified version of adaptive thresholding
    threshold = np.mean(signal)
    return np.maximum(signal - threshold, 0)


def calculate_acf(signal):
    acf = np.correlate(signal, signal, mode='full')[len(signal)-1:]
    acf /= np.max(acf)  # Normalization for better comparison
    return acf

def apply_comb_filter(acf, wv):
    rcf_len = len(wv)
    rcf = np.zeros(rcf_len)
    numelem = 4
    for i in range(2, rcf_len):  # Skipping the first element for indexing reasons
        for a in range(1, numelem + 1):
            for b in range(1 - a, a):
                index = (a * i + b) - 1
                if index < len(acf):
                    rcf[i - 1] += (acf[index] * wv[i - 1]) / (2. * a - 1.)
    return rcf


def normalize_rcf(rcf):
    EPS = 1e-9
    rcf = adaptive_threshold(rcf)
    rcf_sum = np.sum(rcf) + EPS
    return rcf / rcf_sum


def get_rcf(dfframe_in, wv):
    dfframe = adaptive_threshold(dfframe_in)
    acf = calculate_acf(dfframe)
    rcf = apply_comb_filter(acf, wv)
    rcf = normalize_rcf(rcf)
    return rcf

In [ ]:
import numpy as np

wv_len = 128
inputtempo = 120  # Example value, replace as needed
constraintempo = False  # Example value, replace as needed

rayparam = (60 * 44100 / 512) / inputtempo
wv = np.zeros(wv_len)

if constraintempo:
    for i in range(wv_len):
        wv[i] = np.exp(-np.power((i - rayparam), 2) / (2 * np.power(rayparam / 4, 2)))
else:
    for i in range(wv_len):
        wv[i] = (i / np.power(rayparam, 2)) * np.exp(-np.power(i, 2) / (2 * np.power(rayparam, 2)))
        
        


def process_rcf(rcf):
    EPS = 1e-9  # Small constant to avoid division by zero

    # Apply adaptive threshold
    rcf = np.maximum(rcf - np.mean(rcf), 0)

    # Summation with EPS addition
    rcf_sum = np.sum(rcf) + len(rcf) * EPS

    # Normalize to unity
    rcf = (rcf + EPS) / rcf_sum

    return rcf


rcf_processed = process_rcf(rcf)

winlen = 512
step = 128
df_len = len(df)  # Assuming `df` is already defined
rcfmat = []

for i in range(0, df_len - winlen, step):
    dfframe = df[i:i+winlen]
    rcf = get_rcf(dfframe, wv)  # Assuming `get_rcf` function is defined
    rcfmat.append(rcf)
    

